# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt

from api_keys import weather_api_key
from citipy import citipy

In [2]:

#Casts a dataframe with two very long sets of random numbers formatted for use as random lat/long coordinates
cities_df = pd.DataFrame({"long":(np.random.uniform(-1.8,1.8,1200)*100), "lat":(np.random.uniform(-0.9,0.9,1200)*100)})

####
#### API SAVER
####
cities_df = cities_df.truncate(after = 9)


cities_df.head()

#city = citipy.nearest_city(22.99,120.21)
#city.country_code

,long,lat
0,-65.473665,-19.117666
1,172.555839,21.238893
2,111.287716,16.483843
3,-66.431865,76.559592
4,154.534455,-23.195961


## Generate Cities List

In [3]:
cities_name = []
countrycode = []
for i in range(0,len(cities_df['lat'])):
    
    #API saver
    if i>10:
        break
    
    
    lat = cities_df.iloc[i]['lat']
    long = cities_df.iloc[i]['long']
    city = citipy.nearest_city(lat,long)
    
    name = city.city_name
    cities_name.append(name)
    country = city.country_code
    countrycode.append(country)
    
cities_df['City'] = cities_name
cities_df['Country'] = countrycode
cities_df.head(10)

,long,lat,City,Country
0,-65.473665,-19.117666,sucre,bo
1,172.555839,21.238893,butaritari,ki
2,111.287716,16.483843,wanning,cn
3,-66.431865,76.559592,narsaq,gl
4,154.534455,-23.195961,hervey bay,au
5,-56.804168,76.366524,upernavik,gl
6,-115.793113,-64.203493,rikitea,pf
7,-25.932298,-63.089003,chuy,uy
8,-77.525884,-70.933210,ushuaia,ar
9,9.913809,7.064537,wum,cm


In [4]:
#drop duplicate cities in cities_df
cities_copy = cities_df.copy()
new_cities = cities_copy.drop_duplicates("City", keep = 'first')

#reset cities_df's index and truncate at 499 to provide sample size of 500 random cities.
new_cities = new_cities.reset_index()
new_cities = new_cities.truncate( after=499)
new_cities

,index,long,lat,City,Country
0,0,-65.473665,-19.117666,sucre,bo
1,1,172.555839,21.238893,butaritari,ki
2,2,111.287716,16.483843,wanning,cn
3,3,-66.431865,76.559592,narsaq,gl
4,4,154.534455,-23.195961,hervey bay,au
5,5,-56.804168,76.366524,upernavik,gl
6,6,-115.793113,-64.203493,rikitea,pf
7,7,-25.932298,-63.089003,chuy,uy
8,8,-77.525884,-70.933210,ushuaia,ar
9,9,9.913809,7.064537,wum,cm


In [5]:
api_key = weather_api_key


#Establish columns for data to be used in scatter plots
new_cities["Temp"]=""
new_cities["Humidity"]=""
new_cities["Cloudiness"]=""
new_cities["Wind (mph)"]=""
new_cities.head(10)

,index,long,lat,City,Country,Temp,Humidity,Cloudiness,Wind (mph)
0,0,-65.473665,-19.117666,sucre,bo,,,,
1,1,172.555839,21.238893,butaritari,ki,,,,
2,2,111.287716,16.483843,wanning,cn,,,,
3,3,-66.431865,76.559592,narsaq,gl,,,,
4,4,154.534455,-23.195961,hervey bay,au,,,,
5,5,-56.804168,76.366524,upernavik,gl,,,,
6,6,-115.793113,-64.203493,rikitea,pf,,,,
7,7,-25.932298,-63.089003,chuy,uy,,,,
8,8,-77.525884,-70.933210,ushuaia,ar,,,,
9,9,9.913809,7.064537,wum,cm,,,,


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [6]:
#Use iterrows() to loop through the new_cities sample size and perform API calls for each 

In [14]:
#Test API block status

target_url=f"http://api.openweathermap.org/data/2.5/weather?q={new_cities.City[4]}&appid={weather_api_key}"
weather_json = requests.get(target_url).json()
print(weather_json)
#print(weather_api_key)
#print("       ")
#print("       ")
#print(weather_json["main"]["temp"])


#temp_list.append(weather_json["main"]["temp"])
#print(temp_list)

{'coord': {'lon': 152.85, 'lat': -25.3}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'base': 'stations', 'main': {'temp': 289.26, 'feels_like': 289.61, 'temp_min': 289.26, 'temp_max': 289.26, 'pressure': 1017, 'humidity': 93}, 'visibility': 10000, 'wind': {'speed': 1.79, 'deg': 202, 'gust': 2.68}, 'rain': {'1h': 0.86}, 'clouds': {'all': 82}, 'dt': 1599770278, 'sys': {'type': 3, 'id': 2032484, 'country': 'AU', 'sunrise': 1599767422, 'sunset': 1599810030}, 'timezone': 36000, 'id': 2146219, 'name': 'Hervey Bay', 'cod': 200}


In [12]:
temp_list = []
humid_list = []
cloud_list = []
wind_list = []

for index, row in new_cities.iterrows():
    
    
    #API saver
    if index>10:
        break
    
    target_url=f"http://api.openweathermap.org/data/2.5/weather?q={row[3]}&appid={weather_api_key}"
    weather_json = requests.get(target_url).json()
    print(f"Processing Record {index} | {row[3]}")
    
    try:
        #new_cities.set_value(index, "Temp", weather_json["main"]["temp"])
        temp_list.append(weather_json["main"]["temp"])
        humid_list.append(weather_json["main"]["humidity"])
        cloud_list.append(weather_json["clouds"]["all"])
        wind_list.append(weather_json["wind"]["speed"])
        
        
        #add in list appends here for Humidity, Cloudiness, and Wind (mph)
        
        
        
        
        #new_cities.set_value(index, "Humidity", weather_json["main"]["humidity"])
        #new_cities.set_value(index, "Cloudiness", weather_json["clouds"]["all"])
        #new_cities.set_value(index, "Wind (mph)", weather_json["wind"]["speed"])
        
    except:
        temp_list.append("")
        humid_list = ("")
        cloud_list = ("")
        wind_list = ("")
        print("City not found. Skipping...")

Processing Record 0 | sucre
Processing Record 1 | butaritari
Processing Record 2 | wanning
Processing Record 3 | narsaq
Processing Record 4 | hervey bay
Processing Record 5 | upernavik
Processing Record 6 | rikitea
Processing Record 7 | chuy
Processing Record 8 | ushuaia
Processing Record 9 | wum


In [13]:
new_cities["Temp"] = temp_list
new_cities["Humidity"] = humid_list
new_cities["Cloudiness"] = cloud_list
new_cities["Wind (mph)"] = wind_list
###
### ADD A DROPNA STATEMENT DOWN BELOW TO REMOVE ALL MISSING OPTIONS
### OR DELETE BASED ON TEMP BEING "NAN" OR "" OR WHATEVER
### JUST MAKE SURE YOU DO IT
###

#print(len(temp_list))
 

def kelv_to_fahr(temp_kelv):
    """Convert Kelvin to Fahrenheit
    
    Return Fahrenheit conversion of input"""
    
    temp_fahr = (temp_kelv)* 9/5 - 459.67
    return temp_fahr


#converts the strings to numbers so they can be formatted into fahrenheit
#new_cities.dropna() to get rid of the empty variables where data was lacking
#or do a reverse loc statement, remmoving all that has this characteristic - "NaN" or "" 
#before numericization
new_cities["Temp"] = pd.to_numeric(new_cities["Temp"])
new_cities["Temp"] = kelv_to_fahr(new_cities["Temp"])



new_cities.head()

,index,long,lat,City,Country,Temp,Humidity,Cloudiness,Wind (mph)
0,0,-65.473665,-19.117666,sucre,bo,67.676,31,27,7.64
1,1,172.555839,21.238893,butaritari,ki,82.868,77,78,8.99
2,2,111.287716,16.483843,wanning,cn,80.906,84,89,3.88
3,3,-66.431865,76.559592,narsaq,gl,50.000,18,0,6.20
4,4,154.534455,-23.195961,hervey bay,au,60.998,93,82,1.79


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [10]:
#  Get the indices of cities that have humidity over 100%.


In [11]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression